In [3]:
import numpy as np
from tqdm import tqdm

cap1Am = np.load("cap1_Am_nm.npy", allow_pickle = True)
cap1m6Am = np.load("cap1_m6Am_nm.npy", allow_pickle = True)

cap2Am = np.load("cap2_Am_nm.npy", allow_pickle = True)
cap2m6Am = np.load("cap2_m6Am_nm.npy", allow_pickle = True)

pos_count2Am_filt = []
pos_count2m6Am_filt = []

pos_count1Am_filt = []
pos_count1m6Am_filt = []

cap1 = np.concatenate((cap1Am, cap1m6Am))
cap2 = np.concatenate((cap2Am, cap2m6Am))

In [4]:
# Create a dictionary of position information and sequence information for cap1 reads
pos_count2_dups = []
pos_count1_dict = {}
for pos in cap1:
    key = ((pos[4])[:40])
    pos_count1_dict[key] = pos[0:5]

# If a cap2 read matches position and sequence information, it is appended to pos_count2_dups as it is an ambiguous cap1/cap2 read
for seq in tqdm(cap2):
    key = ((seq[4])[:40])
    if key in pos_count1_dict:
        pos_count2_dups.append(seq[0:5])

# A dictionary is made of the position for all ambiguous cap1/cap2 reads found above
pos_count2_dups_more = np.array(pos_count2_dups)     
pos_count1_dups_dict = {}
for pos in pos_count2_dups_more:
    key = tuple(pos[0:4])
    pos_count1_dups_dict[key] = pos[0:5]

# If a read has the same position information as the ambiguous read added to pos_count2_dups, then it is also added to pos_count2_dups so it can be later removed
for seq in tqdm(cap2):
    key = tuple(seq[0:4])
    if key in pos_count1_dups_dict:
        pos_count2_dups.append(seq[0:5])

# A dictionary is made of all reads that should be discarded since they are ambiguous or correspond to the site of an ambiuous read
pos_count2_dups = np.array(pos_count2_dups)
ambAm = []
ambm6Am = []
pos_count2_dups_remove = {}
for pos in pos_count2_dups:
    key = tuple(pos[0:5])
    pos_count2_dups_remove[key] = pos[0:5]

# Those reads that are not in the dictionary of reads to remove are saved as a filtered array of cap2 reads
for seq in tqdm(cap2Am):
    key = tuple(seq[0:5])
    if key not in pos_count2_dups_remove:
        pos_count2Am_filt.append(seq)
    if key in pos_count2_dups_remove:
        ambAm.append(seq)

for seq in tqdm(cap2m6Am):
    key = tuple(seq[0:5])
    if key not in pos_count2_dups_remove:
        pos_count2m6Am_filt.append(seq)
    if key in pos_count2_dups_remove:
        ambm6Am.append(seq)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [5]:
# The same process is repeated for filtering cap1 reads

pos_count1_dups = []

pos_count2_dict = {} 

for pos in cap2:
    key = ((pos[4])[:40])
    pos_count2_dict[key] = pos[0:5]

for seq in tqdm(cap1): 
    key = ((seq[4])[:40])
    if key in pos_count2_dict:
        pos_count1_dups.append(seq[0:5])

pos_count1_dups_more = np.array(pos_count1_dups)     

pos_count2_dups_dict = {}

for pos in pos_count1_dups_more:
    key = tuple(pos[0:4]) 
    pos_count2_dups_dict[key] = pos[0:5]

for seq in tqdm(cap1):
    key = tuple(seq[0:4])
    if key in pos_count2_dups_dict:
        pos_count1_dups.append(seq[0:5]) 

pos_count1_dups = np.array(pos_count1_dups)

pos_count1_dups_remove = {}
for pos in pos_count1_dups:
    key = tuple(pos[0:5])
    pos_count1_dups_remove[key] = pos[0:5]

for seq in tqdm(cap1Am):
    key = tuple(seq[0:5])
    if key not in pos_count1_dups_remove: 
        pos_count1Am_filt.append(seq)

for seq in tqdm(cap1m6Am):
    key = tuple(seq[0:5])
    if key not in pos_count1_dups_remove: 
        pos_count1m6Am_filt.append(seq)

100%|████████████████████████████████| 17141/17141 [00:00<00:00, 1850282.19it/s]


In [6]:
pos_count2Am_filt = np.array(pos_count2Am_filt)
pos_count1Am_filt = np.array(pos_count1Am_filt)
pos_count2m6Am_filt = np.array(pos_count2m6Am_filt)
pos_count1m6Am_filt = np.array(pos_count1m6Am_filt)
ambAm = np.array(ambAm)
ambm6Am = np.array(ambm6Am)

In [7]:
#if not removing ambgiuous reads, save cap1 and cap2 arrays instead
np.save("filtered_cap1_Am.npy", pos_count1Am_filt)
np.save("filtered_cap2_Am.npy", pos_count2Am_filt)
np.save("filtered_cap1_m6Am.npy", pos_count1m6Am_filt)
np.save("filtered_cap2_m6Am.npy", pos_count2m6Am_filt)
np.save("ambiguous_reads_Am.npy", ambAm)
np.save("ambiguous_reads_m6Am.npy", ambm6Am)